In [1]:
from postgres_connect import *
import numpy as np


##### Revenue

In [2]:
#change below for date----------------------------------------------------------------
query = '''
        select awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city from public.revenue_24_jun 
        where attributed_month = '2024-04-01'       
'''
df_revenue = get_df_from_sql('Yes',query,'tunnel-ssh.cer')
df_revenue.head()

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city
0,None,None,None,None,None,None,NaN,NaN,NaN,None
1,GS1976041730,adidas,Delhi,400025,Delivered,Prepaid,500.0,378.0,301.0,Mumbai
2,GS1693678108,adidas,Delhi,560068,Delivered,Prepaid,500.0,37.0,301.0,Bangalore
3,GS1725564044,adidas,Delhi,400078,Delivered,Prepaid,500.0,223.0,301.0,Mumbai
4,GS1141276564,adidas,Delhi,400070,Delivered,Prepaid,500.0,37.0,301.0,Mumbai


In [3]:
df_revenue_filtered = df_revenue[(df_revenue['shipper_id'] == 301) & (df_revenue['shipping_city'] != 'Pune')]
df_revenue_filtered['warehouse_city'] = df_revenue_filtered['warehouse_city'].replace('NCR', 'Delhi')
df_revenue_filtered['shipping_city'] = df_revenue_filtered['warehouse_city'].replace('NCR', 'Delhi')
df_revenue_filtered.groupby('warehouse_city').aggregate('count')

,awb,user_name,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city
warehouse_city,,,,,,,,,
Bangalore,178534,178534,178534,178534,178534,178534,178534,178534,178534
Delhi,183867,183867,183867,183867,183867,183867,183867,183867,183867
Hyderabad,60722,60722,60722,60722,60722,60722,60722,60722,60722
Mumbai,70560,70560,70560,70560,70560,70560,70560,70560,70560


In [4]:
df_revenue_filtered.shape

(493683, 10)

In [5]:
duplicate_revenue = df_revenue_filtered[df_revenue_filtered.duplicated('awb', keep=False)]
duplicate_revenue.head()

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city


In [6]:
df_revenue_filtered.groupby('shipping_city').aggregate('count')

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id
shipping_city,,,,,,,,,
Bangalore,178534,178534,178534,178534,178534,178534,178534,178534,178534
Delhi,183867,183867,183867,183867,183867,183867,183867,183867,183867
Hyderabad,60722,60722,60722,60722,60722,60722,60722,60722,60722
Mumbai,70560,70560,70560,70560,70560,70560,70560,70560,70560


In [7]:
print('Delivered shape',df_revenue_filtered[df_revenue_filtered['shipment_status']=='Delivered'].shape)
print('RTO Delivered shape',df_revenue_filtered[df_revenue_filtered['shipment_status']=='RTO Delivered'].shape)
print('Delivered Revenue',df_revenue_filtered[df_revenue_filtered['shipment_status']=='Delivered']['shipment_charges'].sum())
print('RTO Delivered Revenue',df_revenue_filtered[df_revenue_filtered['shipment_status']=='RTO Delivered']['shipment_charges'].sum())

Delivered shape (460066, 10)
RTO Delivered shape (33617, 10)
Delivered Revenue 29158001.367
RTO Delivered Revenue 3000010.0


In [8]:
df_revenue_delivered = df_revenue_filtered[df_revenue_filtered['shipment_status']=='Delivered']
df_revenue_rto = df_revenue_filtered[df_revenue_filtered['shipment_status']=='RTO Delivered']

In [9]:
df_del_grouped = df_revenue_delivered.groupby(['shipping_city','shipment_status']).aggregate({
    'awb':'count'
}).reset_index().rename(columns={'awb':'orders'})
df_del_grouped.head()

,shipping_city,shipment_status,orders
0,Bangalore,Delivered,168420
1,Delhi,Delivered,169055
2,Hyderabad,Delivered,56635
3,Mumbai,Delivered,65956


In [10]:
df_rto_grouped = df_revenue_rto.groupby(['shipping_city','shipment_status']).aggregate({
    'awb':'count'
}).reset_index().rename(columns={'awb':'orders'})
df_rto_grouped.head()

,shipping_city,shipment_status,orders
0,Bangalore,RTO Delivered,10114
1,Delhi,RTO Delivered,14812
2,Hyderabad,RTO Delivered,4087
3,Mumbai,RTO Delivered,4604


In [11]:
df_rto_per = df_del_grouped.merge(df_rto_grouped,on='shipping_city',how='left')
df_rto_per['rto_per']=df_rto_per['orders_y']/(df_rto_per['orders_x']+df_rto_per['orders_y'])
df_rto_per.head()

,shipping_city,shipment_status_x,orders_x,shipment_status_y,orders_y,rto_per
0,Bangalore,Delivered,168420,RTO Delivered,10114,0.056650
1,Delhi,Delivered,169055,RTO Delivered,14812,0.080558
2,Hyderabad,Delivered,56635,RTO Delivered,4087,0.067307
3,Mumbai,Delivered,65956,RTO Delivered,4604,0.065249


##### Payout

In [14]:
df_rider_cost = pd.read_csv('Rider Payout Sheet All Months - 6_month (7).csv')
df_rider_cost = df_rider_cost.drop(columns=['PFESIC','Finder Fees','Commission','Vendor Pay','MIS PAY'])
df_rider_cost.head()    

,city,rider_id,rider_name,hub,vendor_name,payout_structure,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay,date,Key,exist
0,Bangalore,3637.0,chandu M,DS BLR HBBL,3WH,4 Wheeler,11080.0,0,0.0,0.0,0,11080.0,2024-01-01,Bangalore-45292-3WH,NaN
1,Bangalore,2053.0,DILIP KUMAR TATA ACE,DS BLR MTH,3WH,FIXED,14680.0,1295,0.0,0.0,1295,13385.0,2024-01-01,Bangalore-45292-3WH,NaN
2,Bangalore,2733.0,Manjunath R,DS BLR BOMM,3WH,FIXED,16775.0,0,0.0,0.0,0,16775.0,2024-01-01,Bangalore-45292-3WH,NaN
3,Bangalore,5284.0,Chethu,DS BLR BOMM,3WH,FIXED,4265.0,0,0.0,0.0,0,4265.0,2024-01-01,Bangalore-45292-3WH,NaN
4,Bangalore,2450.0,M Chandu,DS BLR HBBL,3WH,ORDER,5264.0,0,1514.0,0.0,1514,3750.0,2024-01-01,Bangalore-45292-3WH,NaN


In [15]:
#change below for date----------------------------------------------------------------
df_rider_cost_filtered = df_rider_cost[df_rider_cost['date']=='2024-04-01']


##### Vendor

In [16]:
df_vendor_cost = pd.read_csv('Vendor Pay Master - April-Rider-Vendor.csv')
df_vendor_cost.head()

,Key,City,Month,Vendor,Total Pay,Net Deduction,Net Pay,Negative Pay,PFESI,Sourcing Fees,Commission,Comm. Amount,Vendor Pay,MIS PAY,MIS Amount,Dif In MIS,Diff in Invoice,Extra per rs
0,Bangalore-45383-3WH,Bangalore,01/04/2024,3WH,72295,36148.55,36147,₹0,0,0.0,NaN,0,36147,72295.55,0,"-₹72,296",NaN,0.000000
1,Bangalore-45383-Blitz_Self,Bangalore,01/04/2024,Blitz_Self,17600,46690.17,-29091,"₹31,511",0,0.0,NaN,0,"₹31,511","₹78,201",0,"-₹78,201",NaN,1.790398
2,Bangalore-45383-Hindusree,Bangalore,01/04/2024,Hindusree,1443667,246457.57,1197210,₹0,0,0.0,5.0%,"59,861","1,257,071","1,503,528",1503527,-₹1,NaN,0.041464
3,Bangalore-45383-Olive,Bangalore,01/04/2024,Olive,32132,3188.00,28944,₹0,7568,0.0,NaN,1826,38338,41526,38338,"-₹3,188",NaN,0.292357
4,Bangalore-45383-Peel works,Bangalore,01/04/2024,Peel works,"1,151,482",264778.28,886704,"₹1,07,021","₹30,051",12000.0,6%,53202.24,1088978.72,1353757,1353757,₹0,NaN,0.175665


##### FM-MM

In [17]:
df_mm_cost = pd.read_csv('Vendor Pay Master - FM-MM.csv')
df_mm_cost.head()

,Month,Warehouse City,Shipping City,Vendor Name,Cost,Orders (computable),Lane Type
0,01/01/2024,Bangalore,Bangalore,NaN,"₹11,18,555",NaN,Hyperlocal
1,01/01/2024,Delhi,Bangalore,NaN,"₹2,05,699",NaN,Intercity
2,01/01/2024,Delhi,Delhi,NaN,"₹12,91,611",NaN,Hyperlocal
3,01/01/2024,Bangalore,Hyderabad,Rippler,"₹2,94,664",NaN,Intercity
4,01/01/2024,Bangalore,Hyderabad,General Travels,"₹1,03,040",NaN,Intercity


In [18]:
df_mm_cost_filtered = df_mm_cost[df_mm_cost['Month']=='01/04/2024']
def convert_currency_to_number(currency_str):
    if pd.isna(currency_str):
        return None
    return int(currency_str.replace('₹', '').replace(',', ''))

# Apply the function to the Cost column
df_mm_cost_filtered['Cost'] = df_mm_cost_filtered['Cost'].apply(convert_currency_to_number).astype(float)
df_mm_cost_filtered.head()

,Month,Warehouse City,Shipping City,Vendor Name,Cost,Orders (computable),Lane Type
27,01/04/2024,Bangalore,Bangalore,Yellappa,1785760.0,NaN,Hyperlocal
28,01/04/2024,Delhi,Bangalore,Rippler,516988.0,NaN,Intercity
29,01/04/2024,Delhi,Delhi,NaN,1751510.0,NaN,Hyperlocal
30,01/04/2024,Bangalore,Hyderabad,Rippler,712478.0,NaN,Intercity
31,01/04/2024,Delhi,Hyderabad,Rippler,296703.0,NaN,Intercity


##### Fuel

In [19]:
df_fuel_cost = pd.read_csv('Vendor Pay Master - Fuel.csv')
df_fuel_cost.head()

,city,amount,Month
0,Delhi,565626,01/01/2024
1,Bangalore,599032,01/01/2024
2,Hyderabad,352854,01/01/2024
3,Mumbai,379773,01/01/2024
4,Delhi,347268,01/02/2024


In [20]:
df_fuel_cost_filtered = df_fuel_cost[df_fuel_cost['Month']=='01/04/2024']
df_fuel_cost_filtered.head()

,city,amount,Month
31,Delhi,478670,01/04/2024
32,Jaipur,97477,01/04/2024
33,Bangalore,171586,01/04/2024
34,Bangalore,116295,01/04/2024
35,Bangalore,163595,01/04/2024


In [21]:
df_fuel_cost_grouped=df_fuel_cost_filtered.groupby('city').aggregate({'amount':'sum'}).reset_index()
df_fuel_cost_grouped.head()

,city,amount
0,Bangalore,451476
1,Delhi,478670
2,Hyderabad,315798
3,Jaipur,97477
4,Mumbai,420238


##### Lost on Hub

In [22]:
df_hubloss_cost = pd.read_csv('Vendor Pay Master - lostonhub (1).csv')
df_hubloss_cost.head()

,Head,Team,wh_code,Amount,Month
0,Loss on Hub,Bangalore,DS_BLR_MTH,"64,669",01/01/2024
1,Loss on Hub,Mumbai,DS_BOM_KRL,"35,581",01/01/2024
2,Loss on Hub,Hyderabad,DS_HYD_KMP,"37,712",01/01/2024
3,Loss on Hub,Delhi,DS_NCR_DWK,"76,844",01/01/2024
4,Loss on Hub,Bangalore,DS_BLR_MTH,"69,881",01/02/2024


##### Franchise

In [23]:
df_franchise_cost = pd.read_csv('Rider Payout Sheet All Months - Fran_6_month.csv')
df_franchise_cost.head()

,date,city,hub,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay
0,2024-01-01,Bangalore,BLR FRH ECTY,"556,840",0,"18,947",0,"18,947","537,893"
1,2024-01-01,Bangalore,CMRJ-Franchise,"63,640","8,974","14,798",0,"23,771","39,869"
2,2024-01-01,Bangalore,STNG-Franchise,"11,600",0,"1,910",0,"1,910","9,690"
3,2024-01-01,Bangalore,UUTR-Franchise,"325,120",198,"23,478",0,"23,676","301,444"
4,2024-01-01,Delhi,DEL FRH GKP,"667,102","13,645","64,470",0,"78,115","588,987"


In [24]:
df_franchise_cost_filtered = df_franchise_cost[df_franchise_cost['date']=='2024-04-01']
df_franchise_cost_filtered.head()

,date,city,hub,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay
40,2024-04-01,Bangalore,BLDR-Franchise,"632,775",0,"9,444",0,"9,444","623,331"
41,2024-04-01,Bangalore,BLR FRH ECTY,"753,460",0,"10,655","1,000","11,655","741,805"
42,2024-04-01,Bangalore,CMRJ-Franchise,"271,362","8,987","4,369",0,"13,356","258,006"
43,2024-04-01,Bangalore,STNG-Franchise,"191,076",0,"2,283",0,"2,283","188,793"
44,2024-04-01,Bangalore,UUTR-Franchise,"399,376",0,999,0,999,"398,377"


##### Delivered

In [25]:
query_kaptaan = ''' 
    with locus as (
    select * from locus_task_brief where date_trunc('month',dispatch_time)::date = '2024-05-01'
    )
    
    select 
        COALESCE(tour.rider_id, r.rider_id) AS num_rider_id
    ,    COALESCE(t.trip_name, l.awb) as awb 
    ,   t.trip_id
    ,   coalesce(t.start_at,l.dispatch_time) as dispatch_time
    ,   coalesce(n.node_name,nl.node_name) as node_name
    ,   coalesce(n.city_name ,nl.city_name) as shipping_city
    from application_db.trip t
    left join locus as l on l.task_id = t.locus_trip_id
    left join application_db.tour on t.tour_id = tour.tour_id
    left join application_db.node n on tour.node_id = n.node_id
    left join application_db.rider as r on l.rider_id = r.locus_rider_id
    left join application_db.node nl on l.location_id = nl.locus_home_base_id

    where (t.status = 'COMPLETED' or l.status = 'COMPLETED')
    and date_trunc('month',t.start_at)::date = '2024-04-01'
'''

df_delivered_awb = get_df_from_sql('Yes',query_kaptaan,'tunnel-ssh.cer')

df_delivered_awb.head()

,num_rider_id,awb,trip_id,dispatch_time,node_name,shipping_city
0,5341,GS1983755079,3604373,2024-04-02 06:11:54.416,DS HYD MHDP,Hyderabad
1,5341,GS1715658909,3604375,2024-04-02 06:11:42.671,DS HYD MHDP,Hyderabad
2,5341,GS1768566359,3604370,2024-04-02 06:11:45.856,DS HYD MHDP,Hyderabad
3,5341,GS2011607462,3604372,2024-04-02 06:11:30.246,DS HYD MHDP,Hyderabad
4,5341,GS1072179887,3604371,2024-04-02 06:11:50.984,DS HYD MHDP,Hyderabad


In [26]:
# handling duplicate rows
df_delivered_awb['dispatch_time'] = pd.to_datetime(df_delivered_awb['dispatch_time'])
idx = df_delivered_awb.groupby('awb')['dispatch_time'].idxmax()
df_delivered_awb = df_delivered_awb.loc[idx]

In [27]:
df_delivered_awb['node_name'].replace({'FBD-Franchise':'FABD-Franchise'},inplace=True)
df_delivered_awb['node_name'].replace({'DS DEL HKMN':'KTWS-Franchise'},inplace=True)
df_delivered_awb['num_rider_id'].replace({'5473':'6950'},inplace=True)

In [28]:
df_delivered_awb.isna().sum()

num_rider_id     0
awb              0
trip_id          0
dispatch_time    0
node_name        0
shipping_city    0
dtype: int64

In [29]:
df_delivered_awb = df_delivered_awb.dropna()

In [30]:
df_delivered_awb['num_rider_id']=df_delivered_awb['num_rider_id'].astype(int)

In [31]:
df_delivered_city = df_delivered_awb.groupby('shipping_city').aggregate(
    {'awb':'count'
     }).reset_index()
df_delivered_city.head()

,shipping_city,awb
0,Bangalore,153405
1,Delhi,144179
2,Hyderabad,81221
3,Jaipur,10977
4,Mumbai,70289


In [32]:
df_delivered_rider = df_delivered_awb.groupby('num_rider_id').aggregate(
    {'awb':'count'
     }).rename(columns={'awb': 'orders'}).reset_index()
df_delivered_rider.head()

,num_rider_id,orders
0,840,1670
1,845,1750
2,850,1417
3,1000,1701
4,1014,1300


In [33]:
df_delivered_franchise = df_delivered_awb.groupby(['num_rider_id','node_name']).aggregate(
    {'awb':'count'
     }).rename(columns={'awb': 'orders'}).reset_index()
# df_delivered_node_filtered = df_delivered_franchise[df_delivered_franchise['node_name']!='BLR_OFSC']
# df_delivered_node_filtered.head()
df_delivered_franchise.head()

,num_rider_id,node_name,orders
0,840,DS BLR MTH,1670
1,845,DS BLR BOMM,1750
2,850,DS BLR MTH,1417
3,1000,BLDR-Franchise,1701
4,1014,DS BLR BOMM,1300


In [34]:
df_delivered_node = df_delivered_awb.groupby(['node_name']).aggregate(
    {'awb':'count'
     }).rename(columns={'awb': 'orders'}).reset_index()
df_delivered_node_filtered = df_delivered_node[df_delivered_node['node_name']!='BLR_OFSC']
df_delivered_node_filtered.head()
df_delivered_node_filtered.head()

,node_name,orders
0,APAN-Franchise,226
1,BLDR-Franchise,16788
2,BLR FRH ECTY,20310
4,BRBN-Franchise,5562
5,CMRJ-Franchise,6885


In [35]:
df_delivered_node_filtered

,node_name,orders
0,APAN-Franchise,226
1,BLDR-Franchise,16788
2,BLR FRH ECTY,20310
4,BRBN-Franchise,5562
5,CMRJ-Franchise,6885
6,DS BLR BOMM,32269
7,DS BLR HBBL,30774
8,DS BLR MTH,24416
9,DS BOM KDWL,10922
10,DS BOM KRL,24225


##### Rider Pay CPO

In [36]:
df_payout = df_rider_cost_filtered.merge(df_delivered_rider, left_on='rider_id',right_on='num_rider_id', how='left').drop(columns=['num_rider_id'])
df_payout.head()

,city,rider_id,rider_name,hub,vendor_name,payout_structure,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay,date,Key,exist,orders
0,Bangalore,4311.0,Arunkumar,DS BLR HBBL,3WH,ORDER,0.0,0,0.0,0.0,0,0.0,2024-04-01,Bangalore-45383-3WH,NaN,NaN
1,Bangalore,5520.0,chandu raja,DS BLR HBBL,3WH,FIXED,10900.0,6753,0.0,0.0,6753,4148.0,2024-04-01,Bangalore-45383-3WH,NaN,305.0
2,Bangalore,5718.0,sandeep k,DS BLR HBBL,3WH,FIXED,4140.0,897,0.0,0.0,897,3243.0,2024-04-01,Bangalore-45383-3WH,NaN,84.0
3,Bangalore,5446.0,Reehan,DS BLR MTH,3WH,FIXED,2135.0,9547,0.0,0.0,9547,-7412.0,2024-04-01,Bangalore-45383-3WH,NaN,70.0
4,Bangalore,4673.0,SADIQ,DS BLR MTH,3WH,FIXED,21960.0,0,0.0,0.0,0,21960.0,2024-04-01,Bangalore-45383-3WH,NaN,560.0


In [37]:
df_payout['rider_pay_cpo'] = df_payout['total_pay'].astype(float)/df_payout['orders']
df_payout.head()

,city,rider_id,rider_name,hub,vendor_name,payout_structure,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay,date,Key,exist,orders,rider_pay_cpo
0,Bangalore,4311.0,Arunkumar,DS BLR HBBL,3WH,ORDER,0.0,0,0.0,0.0,0,0.0,2024-04-01,Bangalore-45383-3WH,NaN,NaN,NaN
1,Bangalore,5520.0,chandu raja,DS BLR HBBL,3WH,FIXED,10900.0,6753,0.0,0.0,6753,4148.0,2024-04-01,Bangalore-45383-3WH,NaN,305.0,35.737705
2,Bangalore,5718.0,sandeep k,DS BLR HBBL,3WH,FIXED,4140.0,897,0.0,0.0,897,3243.0,2024-04-01,Bangalore-45383-3WH,NaN,84.0,49.285714
3,Bangalore,5446.0,Reehan,DS BLR MTH,3WH,FIXED,2135.0,9547,0.0,0.0,9547,-7412.0,2024-04-01,Bangalore-45383-3WH,NaN,70.0,30.500000
4,Bangalore,4673.0,SADIQ,DS BLR MTH,3WH,FIXED,21960.0,0,0.0,0.0,0,21960.0,2024-04-01,Bangalore-45383-3WH,NaN,560.0,39.214286


##### Franchise Pay CPO

In [38]:
df_franchise_cost_filtered.head()

,date,city,hub,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay
40,2024-04-01,Bangalore,BLDR-Franchise,"632,775",0,"9,444",0,"9,444","623,331"
41,2024-04-01,Bangalore,BLR FRH ECTY,"753,460",0,"10,655","1,000","11,655","741,805"
42,2024-04-01,Bangalore,CMRJ-Franchise,"271,362","8,987","4,369",0,"13,356","258,006"
43,2024-04-01,Bangalore,STNG-Franchise,"191,076",0,"2,283",0,"2,283","188,793"
44,2024-04-01,Bangalore,UUTR-Franchise,"399,376",0,999,0,999,"398,377"


In [39]:
df_delivered_node_filtered.head()

,node_name,orders
0,APAN-Franchise,226
1,BLDR-Franchise,16788
2,BLR FRH ECTY,20310
4,BRBN-Franchise,5562
5,CMRJ-Franchise,6885


In [40]:
df_franchise_payout=df_franchise_cost_filtered.merge(df_delivered_node_filtered,right_on='node_name',left_on='hub',how='left')[['node_name','orders','total_pay']]
df_franchise_payout['vendor_name'] = df_franchise_payout['node_name']
df_franchise_payout['payout_structure'] = 'ORDER'
df_franchise_payout['rider_pay_cpo'] = df_franchise_payout['total_pay'].str.replace(',', '').astype(float) / df_franchise_payout['orders']
df_franchise_payout['Key'] = df_franchise_payout['node_name']

df_franchise_payout.head()

,node_name,orders,total_pay,vendor_name,payout_structure,rider_pay_cpo,Key
0,BLDR-Franchise,16788,"632,775",BLDR-Franchise,ORDER,37.692102,BLDR-Franchise
1,BLR FRH ECTY,20310,"753,460",BLR FRH ECTY,ORDER,37.097981,BLR FRH ECTY
2,CMRJ-Franchise,6885,"271,362",CMRJ-Franchise,ORDER,39.413508,CMRJ-Franchise
3,STNG-Franchise,4852,"191,076",STNG-Franchise,ORDER,39.380874,STNG-Franchise
4,UUTR-Franchise,10112,"399,376",UUTR-Franchise,ORDER,39.495253,UUTR-Franchise


##### combining both and making rider pay

In [41]:
df_cost_master = df_delivered_awb.merge(df_payout,left_on = 'num_rider_id',right_on = 'rider_id',how='left')[['num_rider_id','rider_id','awb','trip_id','node_name','shipping_city','vendor_name','payout_structure','rider_pay_cpo','Key','dispatch_time']]
df_cost_master.head()

,num_rider_id,rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,Key,dispatch_time
0,5194,NaN,BZPPL00093131,3582613,STNG-Franchise,Bangalore,NaN,NaN,NaN,NaN,2024-04-01 05:37:24.488
1,5112,5112.0,BZPPL00106754,3629168,DS HYD TRNK,Hyderabad,Peel works,FIXED,54.897959,Hyderabad-45383-Peel works,2024-04-03 11:04:37.650
2,2776,2776.0,BZPPL00112314,3810901,DS HYD MHDP,Hyderabad,Shiva Shakti,ORDER,36.961131,Hyderabad-45383-Shiva Shakti,2024-04-13 05:54:34.224
3,4461,NaN,BZPPL00118912,3674677,YLHK-Franchise,Bangalore,NaN,NaN,NaN,NaN,2024-04-06 05:06:52.942
4,5050,NaN,BZPPL00132202,3668093,BRBN-Franchise,Hyderabad,NaN,NaN,NaN,NaN,2024-04-05 15:13:49.086


In [42]:
df_cost_master_final = df_cost_master.merge(df_franchise_payout, on='node_name', how='left', suffixes=('', '_fill')).drop(columns=['orders','rider_id','total_pay'])

# Fill the NaN values in df_cost_master with the corresponding values from df_fill
columns_to_fill = ['vendor_name', 'payout_structure', 'rider_pay_cpo', 'Key']
for column in columns_to_fill:
    df_cost_master_final[column] = df_cost_master_final[column].fillna(df_cost_master_final[f"{column}_fill"])
df_cost_master_final = df_cost_master_final.drop(columns=[f"{column}_fill" for column in columns_to_fill])

df_cost_master_final.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,Key,dispatch_time
0,5194,BZPPL00093131,3582613,STNG-Franchise,Bangalore,STNG-Franchise,ORDER,39.380874,STNG-Franchise,2024-04-01 05:37:24.488
1,5112,BZPPL00106754,3629168,DS HYD TRNK,Hyderabad,Peel works,FIXED,54.897959,Hyderabad-45383-Peel works,2024-04-03 11:04:37.650
2,2776,BZPPL00112314,3810901,DS HYD MHDP,Hyderabad,Shiva Shakti,ORDER,36.961131,Hyderabad-45383-Shiva Shakti,2024-04-13 05:54:34.224
3,4461,BZPPL00118912,3674677,YLHK-Franchise,Bangalore,YLHK-Franchise,ORDER,39.126529,YLHK-Franchise,2024-04-06 05:06:52.942
4,5050,BZPPL00132202,3668093,BRBN-Franchise,Hyderabad,BRBN-Franchise,ORDER,38.305645,BRBN-Franchise,2024-04-05 15:13:49.086


In [43]:
df_cost_master_final.isna().sum()

num_rider_id          0
awb                   0
trip_id               0
node_name             0
shipping_city         0
vendor_name         209
payout_structure    209
rider_pay_cpo       209
Key                 209
dispatch_time         0
dtype: int64

In [44]:
df_cost_master_final[df_cost_master_final['vendor_name'].isna()].groupby(['num_rider_id','node_name']).aggregate({'awb':'count'})

awb
num_rider_id node_name       
2716         BLR_OFSC      17
3359         BLR_OFSC       7
             GSCAN TEST     3
5473         DS BOM KDWL  157
5741         BLR_OFSC      17
5767         BLR_OFSC       8

In [45]:
df_cost_master_final.dropna(inplace=True)

In [46]:
duplicate_awb = df_cost_master_final[df_cost_master_final.duplicated('awb', keep=False)]
print(duplicate_awb.shape)
print(duplicate_awb)

(0, 10)
Empty DataFrame
Columns: [num_rider_id, awb, trip_id, node_name, shipping_city, vendor_name, payout_structure, rider_pay_cpo, Key, dispatch_time]
Index: []


##### Vendor Pay CPO

In [47]:
df_vendor_cost_filtered = df_vendor_cost[['Key','Extra per rs']]
df_cost_master2 = df_cost_master_final.merge(df_vendor_cost_filtered,on='Key',how='left')
df_cost_master2.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,Key,dispatch_time,Extra per rs
0,5194,BZPPL00093131,3582613,STNG-Franchise,Bangalore,STNG-Franchise,ORDER,39.380874,STNG-Franchise,2024-04-01 05:37:24.488,NaN
1,5112,BZPPL00106754,3629168,DS HYD TRNK,Hyderabad,Peel works,FIXED,54.897959,Hyderabad-45383-Peel works,2024-04-03 11:04:37.650,0.201697
2,2776,BZPPL00112314,3810901,DS HYD MHDP,Hyderabad,Shiva Shakti,ORDER,36.961131,Hyderabad-45383-Shiva Shakti,2024-04-13 05:54:34.224,0.045151
3,4461,BZPPL00118912,3674677,YLHK-Franchise,Bangalore,YLHK-Franchise,ORDER,39.126529,YLHK-Franchise,2024-04-06 05:06:52.942,NaN
4,5050,BZPPL00132202,3668093,BRBN-Franchise,Hyderabad,BRBN-Franchise,ORDER,38.305645,BRBN-Franchise,2024-04-05 15:13:49.086,NaN


In [48]:
df_cost_master2['vendor_cpo']=df_cost_master2['rider_pay_cpo']*df_cost_master2['Extra per rs']
df_cost_master2.drop(columns=['Extra per rs','Key'],inplace=True)
df_cost_master2.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo
0,5194,BZPPL00093131,3582613,STNG-Franchise,Bangalore,STNG-Franchise,ORDER,39.380874,2024-04-01 05:37:24.488,NaN
1,5112,BZPPL00106754,3629168,DS HYD TRNK,Hyderabad,Peel works,FIXED,54.897959,2024-04-03 11:04:37.650,11.072761
2,2776,BZPPL00112314,3810901,DS HYD MHDP,Hyderabad,Shiva Shakti,ORDER,36.961131,2024-04-13 05:54:34.224,1.668828
3,4461,BZPPL00118912,3674677,YLHK-Franchise,Bangalore,YLHK-Franchise,ORDER,39.126529,2024-04-06 05:06:52.942,NaN
4,5050,BZPPL00132202,3668093,BRBN-Franchise,Hyderabad,BRBN-Franchise,ORDER,38.305645,2024-04-05 15:13:49.086,NaN


In [49]:
df_cost_master2.isna().sum()

num_rider_id             0
awb                      0
trip_id                  0
node_name                0
shipping_city            0
vendor_name              0
payout_structure         0
rider_pay_cpo            0
dispatch_time            0
vendor_cpo          121318
dtype: int64

In [50]:
df_cost_master2=df_cost_master2.fillna('0')

In [51]:
df_cost_master2.isna().sum()

num_rider_id        0
awb                 0
trip_id             0
node_name           0
shipping_city       0
vendor_name         0
payout_structure    0
rider_pay_cpo       0
dispatch_time       0
vendor_cpo          0
dtype: int64

In [52]:
duplicate_awb = df_cost_master2[df_cost_master2.duplicated('awb', keep=False)]
print(duplicate_awb)

Empty DataFrame
Columns: [num_rider_id, awb, trip_id, node_name, shipping_city, vendor_name, payout_structure, rider_pay_cpo, dispatch_time, vendor_cpo]
Index: []


##### Fuel CPO

In [53]:

df_fuel_pay = df_delivered_city.merge(df_fuel_cost_grouped,left_on='shipping_city',right_on='city',how='left')
df_fuel_pay.head()

,shipping_city,awb,city,amount
0,Bangalore,153405,Bangalore,451476
1,Delhi,144179,Delhi,478670
2,Hyderabad,81221,Hyderabad,315798
3,Jaipur,10977,Jaipur,97477
4,Mumbai,70289,Mumbai,420238


In [54]:
df_fuel_pay['fuel_cpo']=df_fuel_pay['amount']/df_fuel_pay['awb']
df_fuel_pay.drop(columns=['amount','awb','city'],inplace=True)

In [55]:
df_cost_master3 = df_cost_master2.merge(df_fuel_pay,on='shipping_city',how='left')
df_cost_master3.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo
0,5194,BZPPL00093131,3582613,STNG-Franchise,Bangalore,STNG-Franchise,ORDER,39.380874,2024-04-01 05:37:24.488,0,2.943033
1,5112,BZPPL00106754,3629168,DS HYD TRNK,Hyderabad,Peel works,FIXED,54.897959,2024-04-03 11:04:37.650,11.072761,3.888132
2,2776,BZPPL00112314,3810901,DS HYD MHDP,Hyderabad,Shiva Shakti,ORDER,36.961131,2024-04-13 05:54:34.224,1.668828,3.888132
3,4461,BZPPL00118912,3674677,YLHK-Franchise,Bangalore,YLHK-Franchise,ORDER,39.126529,2024-04-06 05:06:52.942,0,2.943033
4,5050,BZPPL00132202,3668093,BRBN-Franchise,Hyderabad,BRBN-Franchise,ORDER,38.305645,2024-04-05 15:13:49.086,0,3.888132


In [56]:
df_cost_master3.isna().sum()

num_rider_id        0
awb                 0
trip_id             0
node_name           0
shipping_city       0
vendor_name         0
payout_structure    0
rider_pay_cpo       0
dispatch_time       0
vendor_cpo          0
fuel_cpo            0
dtype: int64

In [57]:
df_cost_master3=df_cost_master3.fillna(0)

In [58]:
df_cost_master3.shape

(459862, 11)

##### Merging with revenue

Note we need to attribute fm-mm cost to RTO Delivered also, so we need to get RTO orders from revenue

In [59]:
df_revenue_filtered.head()

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city
1,GS1976041730,adidas,Delhi,400025,Delivered,Prepaid,500.0,378.0,301.0,Delhi
2,GS1693678108,adidas,Delhi,560068,Delivered,Prepaid,500.0,37.0,301.0,Delhi
3,GS1725564044,adidas,Delhi,400078,Delivered,Prepaid,500.0,223.0,301.0,Delhi
4,GS1141276564,adidas,Delhi,400070,Delivered,Prepaid,500.0,37.0,301.0,Delhi
5,GS1297246637,adidas,Delhi,400016,Delivered,Prepaid,500.0,37.0,301.0,Delhi


In [60]:
df_cost_master3.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo
0,5194,BZPPL00093131,3582613,STNG-Franchise,Bangalore,STNG-Franchise,ORDER,39.380874,2024-04-01 05:37:24.488,0,2.943033
1,5112,BZPPL00106754,3629168,DS HYD TRNK,Hyderabad,Peel works,FIXED,54.897959,2024-04-03 11:04:37.650,11.072761,3.888132
2,2776,BZPPL00112314,3810901,DS HYD MHDP,Hyderabad,Shiva Shakti,ORDER,36.961131,2024-04-13 05:54:34.224,1.668828,3.888132
3,4461,BZPPL00118912,3674677,YLHK-Franchise,Bangalore,YLHK-Franchise,ORDER,39.126529,2024-04-06 05:06:52.942,0,2.943033
4,5050,BZPPL00132202,3668093,BRBN-Franchise,Hyderabad,BRBN-Franchise,ORDER,38.305645,2024-04-05 15:13:49.086,0,3.888132


In [61]:
duplicate_awb = df_cost_master3[df_cost_master3.duplicated('awb', keep=False)]
print(duplicate_awb)

Empty DataFrame
Columns: [num_rider_id, awb, trip_id, node_name, shipping_city, vendor_name, payout_structure, rider_pay_cpo, dispatch_time, vendor_cpo, fuel_cpo]
Index: []


In [62]:
# df_margin = df_revenue_filtered.merge(df_cost_master3,on='awb',how='outer')
df_margin = df_revenue_filtered.merge(df_cost_master3,on='awb',how='outer').drop(columns=['shipping_city_y']).rename(columns={'shipping_city_x':'shipping_city'})
df_margin.head()

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city,num_rider_id,trip_id,node_name,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo
0,BZPPL00093131,Purplle,Bangalore,560027,Delivered,Prepaid,550.0,35.0,301.0,Bangalore,5194.0,3582613.0,STNG-Franchise,STNG-Franchise,ORDER,39.380874,2024-04-01 05:37:24.488,0,2.943033
1,BZPPL00106754,Purplle,Hyderabad,500009,Delivered,COD,540.0,35.0,301.0,Hyderabad,5112.0,3629168.0,DS HYD TRNK,Peel works,FIXED,54.897959,2024-04-03 11:04:37.650,11.072761,3.888132
2,BZPPL00112314,Purplle,Hyderabad,500073,Delivered,COD,975.0,40.0,301.0,Hyderabad,2776.0,3810901.0,DS HYD MHDP,Shiva Shakti,ORDER,36.961131,2024-04-13 05:54:34.224,1.668828,3.888132
3,BZPPL00117900,Purplle,Bangalore,560108,RTO Delivered,COD,260.0,35.0,301.0,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
4,BZPPL00118912,Purplle,Bangalore,560064,Delivered,COD,975.0,35.0,301.0,Bangalore,4461.0,3674677.0,YLHK-Franchise,YLHK-Franchise,ORDER,39.126529,2024-04-06 05:06:52.942,0,2.943033


In [63]:
df_margin.shape

(493685, 19)

In [64]:
duplicate_awb = df_margin[df_margin.duplicated('awb', keep=False)]
print(duplicate_awb)

Empty DataFrame
Columns: [awb, user_name, warehouse_city, drop_pincode, shipment_status, mop, chargeable_weight, shipment_charges, shipper_id, shipping_city, num_rider_id, trip_id, node_name, vendor_name, payout_structure, rider_pay_cpo, dispatch_time, vendor_cpo, fuel_cpo]
Index: []


In [65]:
duplicate_awb.shape

(0, 19)

In [66]:
columns_to_update = [
    'num_rider_id', 'trip_id', 'node_name', 'vendor_name', 
    'payout_structure', 'rider_pay_cpo', 'dispatch_time', 
    'vendor_cpo', 'fuel_cpo'
]
df_margin.loc[df_margin['shipment_status'] == 'RTO Delivered', columns_to_update] = 0

In [67]:
df_margin.isna().sum()

awb                    0
user_name              2
warehouse_city         2
drop_pincode           2
shipment_status        2
mop                    2
chargeable_weight      2
shipment_charges       2
shipper_id             2
shipping_city          2
num_rider_id         208
trip_id              208
node_name            208
vendor_name          208
payout_structure     208
rider_pay_cpo        208
dispatch_time        208
vendor_cpo           208
fuel_cpo             208
dtype: int64

In [68]:
df_margin = df_margin.dropna()

##### FM-MM Costing

In [69]:
df_mm_cost_filtered.head()

,Month,Warehouse City,Shipping City,Vendor Name,Cost,Orders (computable),Lane Type
27,01/04/2024,Bangalore,Bangalore,Yellappa,1785760.0,NaN,Hyperlocal
28,01/04/2024,Delhi,Bangalore,Rippler,516988.0,NaN,Intercity
29,01/04/2024,Delhi,Delhi,NaN,1751510.0,NaN,Hyperlocal
30,01/04/2024,Bangalore,Hyderabad,Rippler,712478.0,NaN,Intercity
31,01/04/2024,Delhi,Hyderabad,Rippler,296703.0,NaN,Intercity


In [70]:
df_mm_cost_filtered['Cost'].sum()

np.float64(6415099.0)

In [71]:
mm_pay=df_mm_cost_filtered.groupby('Shipping City').aggregate({'Cost':'sum'}).reset_index()
mm_pay

,Shipping City,Cost
0,Bangalore,2302748.0
1,Delhi,1751510.0
2,Hyderabad,1272204.0
3,Jaipur,154562.0
4,Mumbai,934075.0


In [72]:
##### Removing 3WH FROM FM-MM
three_wh_pay=df_rider_cost_filtered[df_rider_cost_filtered['vendor_name'] == '3WH'].groupby('city').aggregate({'total_pay':'sum'}).reset_index()
mm_pay = mm_pay.merge(three_wh_pay,left_on='Shipping City',right_on='city',how='left').drop(columns=['city']).fillna(0)
mm_pay['Cost'] = mm_pay['Cost']-mm_pay['total_pay'].astype(float)
mm_pay.drop(columns=['total_pay'],inplace=True)


##### starting rto delivered fm

In [73]:
mm_payout_rto = mm_pay.merge(df_rto_per,right_on='shipping_city',left_on='Shipping City',how='left')
mm_payout_rto.head()

,Shipping City,Cost,shipping_city,shipment_status_x,orders_x,shipment_status_y,orders_y,rto_per
0,Bangalore,2230453.0,Bangalore,Delivered,168420.0,RTO Delivered,10114.0,0.056650
1,Delhi,1751510.0,Delhi,Delivered,169055.0,RTO Delivered,14812.0,0.080558
2,Hyderabad,1256259.0,Hyderabad,Delivered,56635.0,RTO Delivered,4087.0,0.067307
3,Jaipur,154562.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Mumbai,934075.0,Mumbai,Delivered,65956.0,RTO Delivered,4604.0,0.065249


In [74]:
mm_payout_rto['r_factor']=mm_payout_rto['Cost']/(mm_payout_rto['orders_x']+2*mm_payout_rto['orders_y']).astype(float)
mm_payout_rto['r_factor'] = mm_payout_rto['r_factor'].round(2)
mm_payout_rto.head()

,Shipping City,Cost,shipping_city,shipment_status_x,orders_x,shipment_status_y,orders_y,rto_per,r_factor
0,Bangalore,2230453.0,Bangalore,Delivered,168420.0,RTO Delivered,10114.0,0.056650,11.82
1,Delhi,1751510.0,Delhi,Delivered,169055.0,RTO Delivered,14812.0,0.080558,8.82
2,Hyderabad,1256259.0,Hyderabad,Delivered,56635.0,RTO Delivered,4087.0,0.067307,19.38
3,Jaipur,154562.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mumbai,934075.0,Mumbai,Delivered,65956.0,RTO Delivered,4604.0,0.065249,12.43


In [75]:
margin2 = df_margin.merge(mm_payout_rto[['shipping_city','r_factor']], on='shipping_city',how='left').reset_index()

In [76]:
margin2['fm_mm_cpo'] = np.where(
    margin2['shipment_status'] == 'RTO Delivered',
    2 * margin2['r_factor'],
    np.where(
        margin2['shipment_status'] == 'Delivered',
        margin2['r_factor'],
        0  # Or any default value you prefer
    )
)
margin2.head()

,index,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,...,trip_id,node_name,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo,r_factor,fm_mm_cpo
0,0,BZPPL00093131,Purplle,Bangalore,560027,Delivered,Prepaid,550.0,35.0,301.0,...,3582613.0,STNG-Franchise,STNG-Franchise,ORDER,39.380874,2024-04-01 05:37:24.488000,0,2.943033,11.82,11.82
1,1,BZPPL00106754,Purplle,Hyderabad,500009,Delivered,COD,540.0,35.0,301.0,...,3629168.0,DS HYD TRNK,Peel works,FIXED,54.897959,2024-04-03 11:04:37.650000,11.072761,3.888132,19.38,19.38
2,2,BZPPL00112314,Purplle,Hyderabad,500073,Delivered,COD,975.0,40.0,301.0,...,3810901.0,DS HYD MHDP,Shiva Shakti,ORDER,36.961131,2024-04-13 05:54:34.224000,1.668828,3.888132,19.38,19.38
3,3,BZPPL00117900,Purplle,Bangalore,560108,RTO Delivered,COD,260.0,35.0,301.0,...,0.0,0,0,0,0.000000,0,0,0.000000,11.82,23.64
4,4,BZPPL00118912,Purplle,Bangalore,560064,Delivered,COD,975.0,35.0,301.0,...,3674677.0,YLHK-Franchise,YLHK-Franchise,ORDER,39.126529,2024-04-06 05:06:52.942000,0,2.943033,11.82,11.82


In [77]:
margin2[margin2['shipment_status']=='RTO Delivered']['vendor_name'] = 'RTO Delivered'

In [78]:
margin2.isna().sum()

index                0
awb                  0
user_name            0
warehouse_city       0
drop_pincode         0
shipment_status      0
mop                  0
chargeable_weight    0
shipment_charges     0
shipper_id           0
shipping_city        0
num_rider_id         0
trip_id              0
node_name            0
vendor_name          0
payout_structure     0
rider_pay_cpo        0
dispatch_time        0
vendor_cpo           0
fuel_cpo             0
r_factor             0
fm_mm_cpo            0
dtype: int64

In [79]:
margin3 = margin2.drop(columns=['index','user_name','chargeable_weight','shipper_id','r_factor'])
margin3.rename(columns={
    'shipment_charges':'rpo','payout_structure':'rider_payout_structure','fuel_cpo':'rider_fuel_cpo'
},inplace = True)
margin3.head()

,awb,warehouse_city,drop_pincode,shipment_status,mop,rpo,shipping_city,num_rider_id,trip_id,node_name,vendor_name,rider_payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,rider_fuel_cpo,fm_mm_cpo
0,BZPPL00093131,Bangalore,560027,Delivered,Prepaid,35.0,Bangalore,5194.0,3582613.0,STNG-Franchise,STNG-Franchise,ORDER,39.380874,2024-04-01 05:37:24.488000,0,2.943033,11.82
1,BZPPL00106754,Hyderabad,500009,Delivered,COD,35.0,Hyderabad,5112.0,3629168.0,DS HYD TRNK,Peel works,FIXED,54.897959,2024-04-03 11:04:37.650000,11.072761,3.888132,19.38
2,BZPPL00112314,Hyderabad,500073,Delivered,COD,40.0,Hyderabad,2776.0,3810901.0,DS HYD MHDP,Shiva Shakti,ORDER,36.961131,2024-04-13 05:54:34.224000,1.668828,3.888132,19.38
3,BZPPL00117900,Bangalore,560108,RTO Delivered,COD,35.0,Bangalore,0.0,0.0,0,0,0,0.000000,0,0,0.000000,23.64
4,BZPPL00118912,Bangalore,560064,Delivered,COD,35.0,Bangalore,4461.0,3674677.0,YLHK-Franchise,YLHK-Franchise,ORDER,39.126529,2024-04-06 05:06:52.942000,0,2.943033,11.82


In [80]:
margin3.to_csv('margin_april.csv')

##### Vendor Level Margin

In [81]:
vendor_margins=margin3.groupby(['shipping_city','vendor_name','num_rider_id']).aggregate({
    'awb': 'count',
    'rider_pay_cpo': 'sum',
    'vendor_cpo': 'sum',
    'rider_fuel_cpo': 'sum',
    'fm_mm_cpo': 'sum',
    'rpo':'sum'
}).reset_index()
vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay_cpo,vendor_cpo,rider_fuel_cpo,fm_mm_cpo,rpo
0,Bangalore,0,0.0,10114,0.000000,0,0.000000,239094.96,835773.0
1,Bangalore,3WH,1977.0,19,1611.462766,0.0,73.874515,224.58,6871.0
2,Bangalore,3WH,4673.0,468,18352.285714,0.0,1377.339513,5531.76,50709.0
3,Bangalore,3WH,5204.0,547,16830.096000,0.0,1609.839132,6465.54,65106.0
4,Bangalore,3WH,5446.0,54,1647.000000,0.0,158.923790,638.28,7871.0


In [82]:
vendor_margins.dtypes

shipping_city      object
vendor_name        object
num_rider_id      float64
awb                 int64
rider_pay_cpo     float64
vendor_cpo         object
rider_fuel_cpo    float64
fm_mm_cpo         float64
rpo               float64
dtype: object

In [83]:
vendor_margins['vendor_cpo'] = vendor_margins['vendor_cpo'].astype(float)
vendor_margins['fm_mm_cpo'] = vendor_margins['fm_mm_cpo'].astype(float)

In [84]:

vendor_margins['total_cost']=vendor_margins['rider_pay_cpo']+vendor_margins['vendor_cpo']+vendor_margins['rider_fuel_cpo']+vendor_margins['fm_mm_cpo']
vendor_margins['margin']=vendor_margins['rpo']-vendor_margins['total_cost']
vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay_cpo,vendor_cpo,rider_fuel_cpo,fm_mm_cpo,rpo,total_cost,margin
0,Bangalore,0,0.0,10114,0.000000,0.0,0.000000,239094.96,835773.0,239094.960000,596678.040000
1,Bangalore,3WH,1977.0,19,1611.462766,0.0,73.874515,224.58,6871.0,1909.917281,4961.082719
2,Bangalore,3WH,4673.0,468,18352.285714,0.0,1377.339513,5531.76,50709.0,25261.385227,25447.614773
3,Bangalore,3WH,5204.0,547,16830.096000,0.0,1609.839132,6465.54,65106.0,24905.475132,40200.524868
4,Bangalore,3WH,5446.0,54,1647.000000,0.0,158.923790,638.28,7871.0,2444.203790,5426.796210


In [85]:
vendor_margins.rename(columns={'fuel_cpo':'fuel_pay','mm_cpo':'mm_pay','vendor_cpo':'vendor_pay','rider_pay_cpo':'rider_pay'},inplace=True)

vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay,vendor_pay,rider_fuel_cpo,fm_mm_cpo,rpo,total_cost,margin
0,Bangalore,0,0.0,10114,0.000000,0.0,0.000000,239094.96,835773.0,239094.960000,596678.040000
1,Bangalore,3WH,1977.0,19,1611.462766,0.0,73.874515,224.58,6871.0,1909.917281,4961.082719
2,Bangalore,3WH,4673.0,468,18352.285714,0.0,1377.339513,5531.76,50709.0,25261.385227,25447.614773
3,Bangalore,3WH,5204.0,547,16830.096000,0.0,1609.839132,6465.54,65106.0,24905.475132,40200.524868
4,Bangalore,3WH,5446.0,54,1647.000000,0.0,158.923790,638.28,7871.0,2444.203790,5426.796210


In [86]:
vendor_margins.to_csv('vendor_margins_april.csv')

In [87]:
vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay,vendor_pay,rider_fuel_cpo,fm_mm_cpo,rpo,total_cost,margin
0,Bangalore,0,0.0,10114,0.000000,0.0,0.000000,239094.96,835773.0,239094.960000,596678.040000
1,Bangalore,3WH,1977.0,19,1611.462766,0.0,73.874515,224.58,6871.0,1909.917281,4961.082719
2,Bangalore,3WH,4673.0,468,18352.285714,0.0,1377.339513,5531.76,50709.0,25261.385227,25447.614773
3,Bangalore,3WH,5204.0,547,16830.096000,0.0,1609.839132,6465.54,65106.0,24905.475132,40200.524868
4,Bangalore,3WH,5446.0,54,1647.000000,0.0,158.923790,638.28,7871.0,2444.203790,5426.796210


In [88]:
vendor_margins['awb'].sum()

np.int64(493475)

##### Hub Losss Total

In [89]:
df_hubloss_cost_filtered = df_hubloss_cost[df_hubloss_cost['Month']=='01/04/2024']
df_hubloss_cost_filtered['Amount']=df_hubloss_cost_filtered['Amount'].str.replace(',','').astype(float)
df_hubloss_cost_filtered

,Head,Team,wh_code,Amount,Month
19,Loss on Hub,Delhi,DS_NCR_RNGR,10232.0,01/04/2024
20,Loss on Hub,Bangalore,DS_BLR_MTH,4096.0,01/04/2024
21,Loss on Hub,Bangalore,DS_BLR_BOMM,2618.0,01/04/2024
22,Loss on Hub,Mumbai,DS_BOM_KDWL,28625.0,01/04/2024
23,Loss on Hub,Mumbai,DS_BOM_KRL,14773.0,01/04/2024
24,Loss on Hub,Hyderabad,DS_HYD_MHDP,1698.0,01/04/2024
25,Loss on Hub,Hyderabad,DS_HYD_SRNG,53083.0,01/04/2024
26,Loss on Hub,Delhi,DS_NCR_DWK,34797.0,01/04/2024
27,Loss on Hub,Delhi,DS_NCR_HKMN,18753.0,01/04/2024
28,Loss on Hub,Delhi,DS_NCR_RNGR,4681.0,01/04/2024


In [90]:
df_hubloss_cost_filtered.groupby('Team').aggregate({
    'Amount':'sum'
})

,Amount
Team,
Bangalore,6714.0
Delhi,72190.0
Hyderabad,54781.0
Mumbai,43398.0
